In [51]:
import biovec
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [2]:
# Generating corpus file from fasta file
'''pv = biovec.ProtVec("uniprot_sprot.fasta", 
                    n=3,
                    size=100,
                    window=5,
                    sg=1,
                    min_count=2, 
                    out="output_corpusfile_path.txt")'''

'pv = biovec.ProtVec("uniprot_sprot.fasta", \n                    n=3,\n                    size=100,\n                    window=5,\n                    sg=1,\n                    min_count=2, \n                    out="output_corpusfile_path.txt")'

In [3]:
#pv["QAT"]
#pv.to_vecs("ATATQSQSMTEEL")
#pv.save('model_file_path')

#pv2 = biovec.models.load_protvec('model_file_path')

In [52]:
# Prepare dataset for prediction
protein_names, sequences, labels = [], [], []


'''
    Labels:
        "+" stands for "binding protein" => 1
        "-" stands for "non-binding" => 0
'''
def convert_label(label_string):
 
    if label_string == "+":
        return 1
    elif label_string == "-":
        return 0
    else:
        return None

    
# Open file containing dataset    
with open('./ppi_data.fasta') as f:
    lines = f.read().splitlines()
    
    for i in range(len(lines)):
        
        if i % 3 == 0:
            protein_names.append(lines[i])
        elif i % 3 == 1:
            sequences.append(lines[i])
        elif i % 3 == 2:
            labels.append([convert_label(letter) for letter in lines[i]])
        
        # Debug
        if i > 10:
            break
            
            
protein_names = np.array(protein_names)
sequences = np.array(sequences)
labels = np.array(labels)

assert(protein_names.shape[0] == sequences.shape[0] == labels.shape[0])

In [ ]:
# Cross validation in 10-Folds

# !!! Dummy data !!!!
X = np.array([[0,0], [1,1], [2,2], [0,0], [1,1], [2,2], [0,0], [1,1], [2,2], [0,0], [1,1], [2,2], [0,0], [1,1], [2,2],
             [0,0], [1,1], [2,2], [0,0], [1,1], [2,2], [0,0], [1,1], [2,2], [0,0], [1,1], [2,2], [0,0], [1,1], [2,2]])
y = np.array([0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0])

skf = StratifiedKFold(n_splits=10, random_state=42)
skf.get_n_splits(X, y)

for train_index, test_index in skf.split(X, y):
    print("TRAIN indeces:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [19]:
# Define one layer fully connected NN 
'''mlp = MLPClassifier(hidden_layer_sizes=(100),
                    learning_rate_init=0.001,
                    max_iter=200,
                    random_state=42)'''


hyperparameters = {
    'hidden_layer_sizes': [(25), (50), (100)],
    'learning_rate_init': [0.01, 0.001],
    'max_iter': [100, 200, 500]
}

estimator = GridSearchCV(
    MLPClassifier(random_state=42),
    param_grid=hyperparameters, 
    n_jobs=-1)

In [ ]:
#estimator.fit(X_train, y_train)

#predictions = estimator.predict(X_test)

In [ ]:
# TODO
# accuracy, precision, recall, AUC

#accuracy_score(y_true, y_pred)
#precision_score(y_true, y_pred)
#recall_score(y_true, y_pred, average='micro')
#roc_auc_score(y_true, y_scores)